In [18]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch
import pandas as pd
from nn import model, optimizers, losses
from nn.layers import Dense
from nn.activations import Sigmoid, ReLU
class CSVDataset(Dataset):
    def __init__(self, filepath):
        df = pd.read_csv(filepath)
        self.X = df[["x0", "x1"]].values.astype("float32")
        self.y = df["y"].values.astype("int")

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx])
        y = torch.tensor(self.y[idx])
        return x, y
    


dataset = CSVDataset("../data/nn_data.csv")

random_generator = torch.Generator().manual_seed(42)

train_data, test_data = random_split(dataset, [int(0.8*len(dataset)), int(0.2*len(dataset))], generator=random_generator)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, generator=random_generator)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
model = model.Sequential([
    Dense(2, 25),
    ReLU(),
    Dense(25, 10),
    ReLU(),
    Dense(10, 1),
    Sigmoid()
])

loss_fn = losses.BinaryCrossEntropyWithLogits()
optim = optimizers.SGD(model.parameters(), model.gradients(), learning_rate=0.01)
num_epochs = 1000

In [ ]:
"""model = torch.nn.Sequential(
    torch.nn.Linear(2, 25),
    torch.nn.ReLU(),
    torch.nn.Linear(25, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 1),
    torch.nn.Sigmoid()
)"""

In [20]:
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        

        y_pred = model.forward(X_batch.numpy())
        loss = loss_fn.forward(y_pred, y_batch.numpy())

        # Backward pass and optimization
        optim.zero_grad()
        grad = loss_fn.backward()
        model.backward(grad)
        optim.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}")

Epoch [10/1000], Loss: 0.6296
Epoch [20/1000], Loss: 0.4846
Epoch [30/1000], Loss: 0.4745
Epoch [40/1000], Loss: 0.2170
Epoch [50/1000], Loss: 0.2958
Epoch [60/1000], Loss: 0.2753
Epoch [70/1000], Loss: 0.1944
Epoch [80/1000], Loss: 0.4611
Epoch [90/1000], Loss: 0.2148
Epoch [100/1000], Loss: 0.6098
Epoch [110/1000], Loss: 0.4390
Epoch [120/1000], Loss: 0.2359
Epoch [130/1000], Loss: 0.5900
Epoch [140/1000], Loss: 0.4306
Epoch [150/1000], Loss: 0.2569
Epoch [160/1000], Loss: 0.3577
Epoch [170/1000], Loss: 0.5810
Epoch [180/1000], Loss: 0.3620
Epoch [190/1000], Loss: 0.1694
Epoch [200/1000], Loss: 0.2832
Epoch [210/1000], Loss: 0.1442
Epoch [220/1000], Loss: 0.1685
Epoch [230/1000], Loss: 0.2404
Epoch [240/1000], Loss: 0.4116
Epoch [250/1000], Loss: 0.1591
Epoch [260/1000], Loss: 0.4443
Epoch [270/1000], Loss: 0.2425
Epoch [280/1000], Loss: 0.1910
Epoch [290/1000], Loss: 0.3989
Epoch [300/1000], Loss: 0.3401
Epoch [310/1000], Loss: 0.1803
Epoch [320/1000], Loss: 0.2079
Epoch [330/1000],

In [43]:
import numpy as np
model.eval()

correct = 0
total = 0
print("Evaluating on test data...")
print(len(test_loader))
for x, y in test_loader:
    x_np = x.numpy().astype(np.float32)
    y_np = y.numpy()                  # shape (B,)

    logits = model.forward(x_np, training=False)     # shape (B, C)
    pred = (logits >= 0.5).astype(np.float32)                     # shape (B,)
    correct += (pred.squeeze() == y_np).sum()                  # sum over batch -> scalar
    total   += y_np.shape[0]  
    print("y_np.shape:", y_np.shape)   
    print("pred.shape:", pred.shape)
acc = correct / total
print("Test Results:") 
print(f"Correct: {correct}, Total: {total}")
print(f"Accuracy: {acc:.4f}")


Evaluating on test data...
7
y_np.shape: (32,)
pred.shape: (32, 1)
y_np.shape: (32,)
pred.shape: (32, 1)
y_np.shape: (32,)
pred.shape: (32, 1)
y_np.shape: (32,)
pred.shape: (32, 1)
y_np.shape: (32,)
pred.shape: (32, 1)
y_np.shape: (32,)
pred.shape: (32, 1)
y_np.shape: (8,)
pred.shape: (8, 1)
Test Results:
Correct: 188, Total: 200
Accuracy: 0.9400
